**Topic:** Historical Conditional Value at Risk  
**Autor:** George Estrella Valverde  
**Date:** January 12, 2025  
***

In [12]:
import pandas as pd
import numpy as np
import yfinance as yf

In [13]:
def HCVaR(tickers, start_date, end_date, confidence_level):
    """
    Calculate Conditional Historical Value at Risk (CVaR) for given tickers, date range, and confidence level
    across multiple frequencies (daily, weekly, monthly, annual, etc.).
    
    Parameters:
        tickers (list): List of stock tickers.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        confidence_level (float): Confidence level (e.g., 0.95 for 95% confidence).
        
    Returns:
        pd.DataFrame: DataFrame containing Conditional Historical VaR values for each frequency.
    """
    # Download close prices
    prices = yf.download(tickers, start=start_date, end=end_date)['Close']
    if isinstance(prices, pd.Series) and len(tickers) == 1:
        prices = prices.to_frame(name=tickers[0])
    
    # Calculate logarithmic returns
    log_returns = np.log(prices / prices.shift(1))

    # Map frequencies to rolling window sizes
    freq_map = {
        'daily': 1,
        'weekly': 5,
        'biweekly': 10,
        'monthly': 22,
        'quarterly': 63,
        'semiannual': 126,
        'annual': 252
    }
    
    # Initialize the result dictionary
    results = {ticker: {} for ticker in tickers}
    
    # Calculate Conditional VaR (CVaR) for each frequency
    for freq, window in freq_map.items():
        rolling_returns = log_returns.rolling(window=window).sum()
        for ticker in tickers:
            # Calculate Historical VaR (HVaR)
            var_value = np.percentile(rolling_returns[ticker].dropna(), (1 - confidence_level) * 100)
            cvar_value = rolling_returns[ticker][rolling_returns[ticker] <= var_value].mean()
            results[ticker][f"{freq.capitalize()}_HCVaR"] = -round(cvar_value, 4)
    
    # Convert results dictionary to DataFrame
    historicalCVaR_df = pd.DataFrame(results).T.reset_index(names='Ticker')
    historicalCVaR_df = historicalCVaR_df.sort_values(by='Ticker', ascending=True).reset_index(drop=True)
    return historicalCVaR_df

# Ejemplo de uso
tickers = ['AMZN', 'AAPL', 'NVDA', 'INTC', 'META', 'MSFT', 'GOOGL', 'JPM', 'NFLX', 'BAC']
start_date = '2005-01-01'
end_date = '2025-01-03'
confidence_level = 0.995

HCVaR_df = HCVaR(tickers, start_date, end_date, confidence_level)
HCVaR_df

[*********************100%%**********************]  10 of 10 completed


,Ticker,Daily_HCVaR,Weekly_HCVaR,Biweekly_HCVaR,Monthly_HCVaR,Quarterly_HCVaR,Semiannual_HCVaR,Annual_HCVaR
0,AAPL,0.0894,0.1845,0.2619,0.4336,0.6400,0.7180,0.7474
1,AMZN,0.1137,0.2087,0.2911,0.3799,0.6354,0.6163,0.7271
2,BAC,0.1844,0.4345,0.5490,0.7814,1.3794,1.9487,2.2201
3,GOOGL,0.0838,0.1506,0.2089,0.3022,0.5129,0.6714,0.8741
4,INTC,0.1127,0.2294,0.3100,0.4588,0.5356,0.7875,0.8136
5,JPM,0.1254,0.2491,0.3038,0.4125,0.5422,0.6456,0.7060
6,META,0.1331,0.2930,0.3545,0.4413,0.5787,0.7759,1.2041
7,MSFT,0.0821,0.1436,0.1713,0.2325,0.3297,0.4821,0.6207
8,NFLX,0.1738,0.3672,0.4836,0.6710,1.1446,1.3467,1.4535
9,NVDA,0.1368,0.3103,0.4256,0.5824,0.7447,1.1782,1.6102
